In [5]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOHumanoidDR-v5" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 2 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

from helpers.utils import ENVIRONMENTS_MAP, get_algorithms
ALGORITHMS = get_algorithms(ENV_NAME)

### Normalize Front and Calculate Normalized Hypervolume and EUM for Generalist and Specialist

Import helpers

In [6]:
import numpy as np

sys.path.append(os.path.abspath(os.path.join('../..')))

def get_normalized_vec_returns(all_vec_returns, minmax_range):
    minmax_array = np.array([minmax_range[i] for i in range(all_vec_returns.shape[-1])])
    min_vals = minmax_array[:, 0].reshape(1, 1, -1) # reshape to (1, 1, n_objectives) for broadcasting
    max_vals = minmax_array[:, 1].reshape(1, 1, -1)

    clipped_vec_returns = np.clip(all_vec_returns, min_vals, max_vals) # broadcasted clipping
    
    # Normalize
    normalized_vec_returns = (clipped_vec_returns - min_vals) / (max_vals - min_vals)
    
    return normalized_vec_returns

In [7]:
from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100 # CHANGE THIS TO THE NUMBER OF WEIGHTS YOU WANT TO USE, NORMALLY ITS 100, FOR MARIO ITS 32
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 1.]),
 array([0.01006263, 0.98993737]),
 array([0.02011696, 0.97988304]),
 array([0.03015917, 0.96984083]),
 array([0.04025468, 0.95974532])]

### Combine the fronts of all the Specialists for each environment

In [8]:
import warnings
from mo_utils.pareto import filter_pareto_dominated

curr_envs = ENVIRONMENTS_MAP[ENV_NAME]
SPECIALIST_FRONT = "eval/front" # don't change this, this is the discounted fronts but poorly named!!
path_to_find_fronts = f"data/single_env/{SPECIALIST_FRONT}/{ENV_NAME}"

for env in curr_envs:
    unfiltered_combined_front_df = None
    path_to_find_front_for_subenv = path_to_find_fronts + f"/{env}"
    
    for algo in ALGORITHMS:
        if os.path.exists(path_to_find_front_for_subenv + f"/{algo}.csv"):
            front_df = pd.read_csv(path_to_find_front_for_subenv + f"/{algo}.csv")
            print(f"Found front for {env} - {algo}, total row: {len(front_df)}")
            if unfiltered_combined_front_df is None:
                unfiltered_combined_front_df = front_df
            else:
                unfiltered_combined_front_df = pd.concat([unfiltered_combined_front_df, front_df])

    if unfiltered_combined_front_df is None:
        warnings.warn(f"No fronts found for {env}")
        continue
    
    unfiltered_combined_front_df = unfiltered_combined_front_df.reset_index(drop=True)
    print(f"Combined front for {env} has {len(unfiltered_combined_front_df)} rows")

    for column in unfiltered_combined_front_df.columns:
        min_value = unfiltered_combined_front_df[column].min()
        max_value = unfiltered_combined_front_df[column].max()
        print(f"{column}, Min: {min_value}, Max: {max_value}")

    combined_front_array = unfiltered_combined_front_df.to_numpy()
    filtered_combined_front_array = filter_pareto_dominated(combined_front_array)

    combined_front_df = pd.DataFrame(filtered_combined_front_array, columns=unfiltered_combined_front_df.columns)
    print(f"Filtered front for {env} has {len(combined_front_df)} rows")
    save_dir = f"data/single_env/combined_fronts/{ENV_NAME}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    combined_front_df.to_csv(f"{save_dir}/{env}.csv", index=False)

Found front for MOHumanoidDefault-v5 - MORL-D(MOSAC)-SB+PSA, total row: 28
Found front for MOHumanoidDefault-v5 - GPI-LS Continuous Action, total row: 3
Found front for MOHumanoidDefault-v5 - CAPQL, total row: 9
Combined front for MOHumanoidDefault-v5 has 40 rows
objective_1, Min: 216.90978306123483, Max: 395.3616006055876
objective_2, Min: -34.9915380753552, Max: 189.06336671166113
Filtered front for MOHumanoidDefault-v5 has 34 rows
Found front for MOHumanoidLight-v5 - MORL-D(MOSAC)-SB+PSA, total row: 23
Found front for MOHumanoidLight-v5 - CAPQL, total row: 2
Combined front for MOHumanoidLight-v5 has 25 rows
objective_1, Min: 218.1958525922188, Max: 548.3034055981165
objective_2, Min: 66.74609125318608, Max: 156.4717393944678
Filtered front for MOHumanoidLight-v5 has 20 rows
Found front for MOHumanoidHeavy-v5 - MORL-D(MOSAC)-SB+PSA, total row: 17
Found front for MOHumanoidHeavy-v5 - CAPQL, total row: 1
Combined front for MOHumanoidHeavy-v5 has 18 rows
objective_1, Min: 152.1250030001

### Normalize the fronts and calculate normalized hypervolume and EUM for SPECIALIST

In [9]:
from evaluation import get_eval_params

# These should exist in 'experiments/evaluation/eval_params.yaml' for current environment
# For new environments, you can add them to the file using the minmax values from the previous step
normalization_data = get_eval_params(ENV_NAME)['normalization']
normalization_data

{'MOHumanoidDefault-v5': {0: [0, 395.4], 1: [-35, 188]},
 'MOHumanoidLight-v5': {0: [0, 548], 1: [0, 156.4]},
 'MOHumanoidHeavy-v5': {0: [0, 232], 1: [0, 135.4]},
 'MOHumanoidLowDamping-v5': {0: [0, 374], 1: [0, 182.4]},
 'MOHumanoidHard-v5': {0: [0, 330], 1: [-9, 88.6]}}

In [10]:
from helpers.utils import ENVIRONMENTS_MAP

FRONT = "eval/discounted_front" # don't change this, front extracted for specialists are only the discounted ones!!
file_path = f"data/{FRONT}/{ENV_NAME}"
scores_save_path = f"data/scores/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

In [11]:
normalized_specialist_hypervolumes = []
normalized_specialist_eums = []

for env in ENVIRONMENTS_MAP[ENV_NAME]:
    min_max_ranges = normalization_data[env]
    best_env_front_path = f"data/single_env/combined_fronts/{ENV_NAME}/{env}.csv"
    assert os.path.exists(best_env_front_path), f"File {best_env_front_path} does not exist"
    
    best_env_front = pd.read_csv(best_env_front_path)
    data_array = best_env_front.to_numpy()
    normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)
    normalized_specialist_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
    normalized_specialist_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

specialist_data = {f"normalized_hypervolume/{env}": [normalized_specialist_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
specialist_data.update({f"normalized_eum/{env}": [normalized_specialist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
specialist_normalized_hv = pd.DataFrame(specialist_data)
specialist_normalized_hv.to_csv(f"{scores_save_path}/specialist.csv", index=False)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False



### Normalize the fronts and calculate normalized hypervolume and EUM for GENERALIST

In [12]:
# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        normalized_hypervolumes = []
        normalized_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            min_max_ranges = normalization_data[env]
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)

            normalized_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
            normalized_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

        data = {f"normalized_hypervolume/{env}": [normalized_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        data.update({f"normalized_eum/{env}": [normalized_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)
            

# Calculate NHGR

In [14]:
# get the normalized hypervolumes of the specialists
specialist_normalized_hv_data = pd.read_csv(f"{scores_save_path}/specialist.csv")

for algo in ALGORITHMS:
    for seed in SEEDS:
        nhgrs = []
        # get the normalized hypervolumes we extracted earlier
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_normalized_hv_data = pd.read_csv(file)

        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            # Filter columns that start with "normalized_hypervolume"
            col = f"normalized_hypervolume/{env}"

            specialist_normalized_hv = specialist_normalized_hv_data[col].values[0]
            generalist_normalized_hv = seed_normalized_hv_data[col].values[0]
            
            env_nhgr = generalist_normalized_hv / specialist_normalized_hv

            seed_normalized_hv_data[f'NHGR/{env}'] = env_nhgr
        
        seed_normalized_hv_data.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)